## Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import itertools
import dill
import random
import missingno
from rff import RFF
from backtest import Backtest
from backtest_tc import Backtest_tc
from joblib import Parallel, delayed
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import r2_score, precision_score, recall_score, accuracy_score
pd.options.mode.chained_assignment = None  

## Import Data

In [6]:
columns = ["cfabs", "cfcom", "cftw", "ir", "money", "cpi", "indprod"]

nber = pd.read_csv("data/NBER_20210719_cycle_dates_pasted.csv")[1:]
nber["peak"] = pd.to_datetime(nber["peak"])
nber["trough"] = pd.to_datetime(nber["trough"])

data_raw = pd.read_csv("data/mega.csv")
data_raw["yyyymm"] = pd.to_datetime(data_raw["yyyymm"], format='%Y%m', errors='coerce')
data_raw = data_raw.set_index("yyyymm")
data_raw[data_raw.columns] = data_raw[data_raw.columns].astype(float)
data_raw = data_raw.rename({"gbp":"returns"}, axis=1)
data_raw["lag_returns"] = data_raw["returns"].shift()

returns = data_raw["returns"].copy()

data = data_raw[columns].dropna()
returns = returns[returns.index.isin(data.index)]

## Standardise

In [7]:
for col in columns:
    data[col] = (data[col] - data[col].expanding(36).mean())/data[col].expanding(36).std()
returns_std = returns.rolling(12).std().shift()
returns = returns / returns_std

data = data[36:]
returns = returns[36:]

# Simulation

In [18]:
random_lists = []
for a in range(100):
    random_walk = list()
    timing_strategy = list()
    random_walk.append(0)
    for i in range(1, len(returns)-12):
        value = random_walk[i-1]+np.random.normal(0,1)
        random_walk.append(value)
    random_lists.append(random_walk)
    
average_list = []
for i in range(len(random_lists[0])):
    average_value = sum(random_list[i] for random_list in random_lists) / len(random_lists)
    average_list.append(average_value)
random_walk = average_list

for b in range(len(random_walk)):
    ts = float(random_walk[b] * returns[b])
    timing_strategy.append(ts)
    
print("R2:", r2_score(returns[12:], random_walk))
# Simulation
#before TCs
mean = (sum(timing_strategy)/len(timing_strategy)) * 12
std = np.std(timing_strategy, axis = 0) * np.sqrt(12)
print("Before TCs er", mean)
print("Before TCs vol", std)
print("Before TCs SR", mean/std)

#after TCs
return_tc = list()
ptc_delta = list()
delta = list()
return_tc.append(0)
for j in range(1,len(timing_strategy)):
    delta = abs(random_walk[j] - random_walk[j-1])
    ptcdelta = delta * 0.01
    return_value = timing_strategy[j] - ptcdelta
    return_tc.append(return_value)
mean_tc = (sum(return_tc)/len(return_tc)) * 12
std_tc = np.std(return_tc, axis = 0) * np.sqrt(12)
print("After TCs er", mean_tc)
print("After TCs vol", std_tc)
print("After TCs SR", mean_tc/std_tc)

R2: -0.6103443756129161
Before TCs er 0.26628478083878604
Before TCs vol 3.9255606065113087
Before TCs SR 0.06783356761760365
After TCs er 0.2574068673473882
After TCs vol 3.925609147968627
After TCs SR 0.0655711910291904
